# MBA Strategy Agent v4 — Interactive Colab Notebook

A multi-agent business strategy assistant using **LangGraph** with models via **OpenRouter**.

**Pipeline:**
```
START → intake → clarify → [Gate 1] → plan_topics →
┌→ research_and_propose ⇄ topic_critic → [Gate 2] → (next topic)─┐
└──────────────────────────────────────────────────────────────────┘
→ synthesizer → action_plan_90d → [Gate 3] → END
```

You will interact with the agent at **3 human gates** — reviewing problem framing,
per-topic research proposals, and the final recommendation report.

### Setup checklist
1. Add your **OPENROUTER_API_KEY** and **TAVILY_API_KEY** to the Colab Secrets sidebar (🔑 icon on the left)
2. Run all cells in order
3. When a gate fires, read the rendered output and type your feedback (or press Enter to approve)

---
## 1. Install Dependencies

In [ ]:
!pip install -q langchain-openai langgraph tavily-python python-dotenv python-docx pyyaml pydantic

---
## 2. API Keys

- `OPENROUTER_API_KEY` — get one at [openrouter.ai](https://openrouter.ai)
- `TAVILY_API_KEY` — get one at [tavily.com](https://tavily.com)

Paste your keys between the quotes below.

In [ ]:
import os

os.environ["OPENROUTER_API_KEY"] = ""  # ← paste your OpenRouter key here
os.environ["TAVILY_API_KEY"]     = ""  # ← paste your Tavily key here

assert os.environ["OPENROUTER_API_KEY"], "⚠ Please fill in your OPENROUTER_API_KEY above"
assert os.environ["TAVILY_API_KEY"],     "⚠ Please fill in your TAVILY_API_KEY above"

print("API keys set.")

---
## 3. Clone Repo & Load Agent

In [ ]:
import os

REPO_URL = "https://github.com/qinazihan/Final_Project_Sp26.git"
REPO_DIR = "/content/Final_Project_Sp26"

if not os.path.exists(REPO_DIR):
    !git clone {REPO_URL} {REPO_DIR}
else:
    print(f"Repo already cloned at {REPO_DIR}")

---
## 4. Configuration

Edit the settings below to customize your run. The config is written to disk
so the agent module can load it at import time.

In [ ]:
import yaml
from pathlib import Path

# ── Edit these settings ──
CONFIG = {
    "input_query": "How should I start a beverage business in Sweden?",

    "openrouter_base_url": "https://openrouter.ai/api/v1",

    "agents": {
        "intake":      {"model": "openai/gpt-4o"},
        "researcher":  {"model": "openai/gpt-4o"},
        "critic":      {"model": "openai/gpt-4o"},
        "synthesizer": {"model": "openai/gpt-4o"},
    },

    "auto_approve": False,
    "max_clarify_rounds": 5,
    "max_research_topics": 4,
    "max_web_search_ct": 10,
    "max_debate_rounds": 10,
    "max_human_revision_on_proposal": 2,
    "max_human_revision_on_plan": 2,
}

# Write config to disk for the agent module
config_path = Path(REPO_DIR) / "MBA-course" / "config.yaml"
with open(config_path, "w") as f:
    yaml.dump(CONFIG, f, default_flow_style=False, sort_keys=False)

print(f"Config written to {config_path}")
print(f"  Query: {CONFIG['input_query']}")
print(f"  Models: {', '.join(v['model'] for v in CONFIG['agents'].values())}")

In [ ]:
import sys

agent_dir = str(Path(REPO_DIR) / "MBA-course")
if agent_dir not in sys.path:
    sys.path.insert(0, agent_dir)

# Import the agent module — this loads config, sets up Tavily, etc.
import mba_agent_v4 as mba

print(f"\nModule loaded: {mba.__file__}")
print(f"Input query:   {mba.INPUT_QUERY}")

---
## 5. Run the Agent (Interactive)

The cell below runs the full pipeline. When the agent reaches a **human gate**,
it will display the content as formatted Markdown and wait for your input.

- **Press Enter** (or type `approve`) to accept and continue
- **Type feedback** to ask the agent to revise

In [ ]:
import time
from datetime import datetime
from IPython.display import display, Markdown, HTML
from langgraph.checkpoint.memory import MemorySaver
from langgraph.types import Command

# ── Build graph ──
agent = mba.build_graph(checkpointer=MemorySaver())
config = {"configurable": {"thread_id": "colab-run-1"}}

# ── Timestamped output dir in working directory ──
ts = datetime.now().strftime("%Y%m%d_%H%M%S")
output_dir = Path(ts)
output_dir.mkdir(parents=True, exist_ok=True)
print(f"Output dir: {output_dir.resolve()}\n")

# ── Run ──
t0 = time.time()
result = agent.invoke({"user_query": mba.INPUT_QUERY}, config)

gate_num = 0
while True:
    snapshot = agent.get_state(config)
    if not snapshot.next:
        break

    # Display interrupt content as rendered Markdown
    for task in snapshot.tasks:
        if hasattr(task, 'interrupts'):
            for intr in task.interrupts:
                gate_num += 1
                display(HTML("<hr style='border:3px solid #2E75B6; margin:24px 0 8px 0'>"))
                display(Markdown(f"### Human Gate {gate_num}"))
                display(Markdown(str(intr.value)))
                display(HTML("<hr style='border:3px solid #2E75B6; margin:8px 0 16px 0'>"))

    feedback = input("approve / or type feedback > ").strip()
    if not feedback:
        feedback = "approved"

    result = agent.invoke(Command(resume=feedback), config)

elapsed = time.time() - t0

display(HTML("<hr style='border:4px solid #1B2A4A; margin:24px 0'>"))
display(Markdown("## Agent Complete"))
print(f"Elapsed: {elapsed:.0f}s")

---
## 6. View Results

In [ ]:
display(Markdown("### Recommendation Report"))
display(Markdown(result.get("recommendation", "_No recommendation generated._")))

display(HTML("<hr>"))

display(Markdown("### 90-Day Action Plan"))
display(Markdown(result.get("action_plan", "_No action plan generated._")))

---
## 7. Save Outputs

Saves Markdown report, DOCX, and metadata to `results/<timestamp>/`.

In [ ]:
import yaml

agents_str = ', '.join(f'{k}={v["model"]}' for k, v in CONFIG["agents"].items())

# ── Markdown ──
md_path = output_dir / f"mba_v4_report_{ts}.md"
with open(md_path, "w") as f:
    f.write(f"# MBA Strategy Report v4\n\n")
    f.write(f"**Generated:** {datetime.now().isoformat()}\n")
    f.write(f"**Agents:** {agents_str}\n\n---\n\n")
    f.write(result.get("recommendation", ""))
    f.write(f"\n\n---\n\n")
    f.write(result.get("action_plan", ""))
print(f"Markdown: {md_path}")

# ── meta.yaml ──
with open(output_dir / "meta.yaml", "w") as f:
    yaml.dump({
        "timestamp": datetime.now().isoformat(),
        "input_query": CONFIG["input_query"],
        "auto_approve": CONFIG["auto_approve"],
        "models": {k: v["model"] for k, v in CONFIG["agents"].items()},
        "elapsed_seconds": round(elapsed, 1),
    }, f, default_flow_style=False, sort_keys=False)
print(f"Meta:     {output_dir / 'meta.yaml'}")

# ── DOCX (optional) ──
try:
    from docx import Document as DocxDocument
    from docx.shared import Pt, Cm

    doc = DocxDocument()
    for section in doc.sections:
        section.top_margin = Cm(2.5)
        section.bottom_margin = Cm(2.5)
        section.left_margin = Cm(2.5)
        section.right_margin = Cm(2.5)

    style = doc.styles["Normal"]
    style.font.name = "Calibri"
    style.font.size = Pt(11)

    doc.add_heading("MBA Strategy Report", level=0)
    doc.add_paragraph(CONFIG["input_query"])
    doc.add_paragraph(f"Generated: {datetime.now().strftime('%B %d, %Y at %H:%M')}")
    doc.add_paragraph(f"Models: {agents_str}")
    doc.add_page_break()

    recommendation = result.get("recommendation", "")
    if recommendation:
        for line in recommendation.split("\n"):
            stripped = line.strip()
            if stripped.startswith("# "):
                doc.add_heading(stripped.lstrip("# "), level=1)
            elif stripped.startswith("## "):
                doc.add_heading(stripped.lstrip("# "), level=2)
            elif stripped.startswith("### "):
                doc.add_heading(stripped.lstrip("# "), level=3)
            elif stripped.startswith("- ") or stripped.startswith("* "):
                doc.add_paragraph(stripped[2:], style="List Bullet")
            elif stripped:
                doc.add_paragraph(stripped)

    doc.add_page_break()

    action_plan = result.get("action_plan", "")
    if action_plan:
        for line in action_plan.split("\n"):
            stripped = line.strip()
            if stripped.startswith("# "):
                doc.add_heading(stripped.lstrip("# "), level=1)
            elif stripped.startswith("## "):
                doc.add_heading(stripped.lstrip("# "), level=2)
            elif stripped.startswith("### "):
                doc.add_heading(stripped.lstrip("# "), level=3)
            elif stripped.startswith("- ") or stripped.startswith("* "):
                doc.add_paragraph(stripped[2:], style="List Bullet")
            elif stripped:
                doc.add_paragraph(stripped)

    docx_path = output_dir / f"mba_v4_report_{ts}.docx"
    doc.save(str(docx_path))
    print(f"DOCX:     {docx_path}")
except ImportError:
    print("[SKIP] python-docx not installed")
except Exception as e:
    print(f"[WARNING] DOCX generation failed: {e}")

print(f"\nAll outputs saved to: {output_dir}")

In [ ]:
# List all saved files
print(f"Contents of {output_dir}:\n")
for p in sorted(output_dir.iterdir()):
    size = p.stat().st_size
    print(f"  {p.name:40s} {size:>8,} bytes")